In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Define a function for taking an integral

In [ ]:
def func(x):
    a = 1.01
    b =-3.04
    c = 2.07
    return a*x**2 + b*x + c


Define it's integral so we know the right answer

In [ ]:
def func_integral(x):
    a = 1.01
    b =-3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x

Define the core of the trapezoid method

In [ ]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))


Define a wrapper function to perform trapezoid method

In [ ]:
def trapezoid_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evaluations to use
    #Note the number of chunks will be N-1
    #so if N is odd, then we don't need to adjust the last segment
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        #print(i,i+1,x[i],x[i]+h,x[-2],x[-1])
        
    #return the answer
    return Fint

Define the core of the simpsons method

In [ ]:
def simpson_core(f,x,h):
    return h*( f(x) + 4*(x+h) + f(x+2*h))/3.

Define a wrapper function to perform Simpson's Method

In [ ]:
def simpsons_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evaluations to use
    #note the number of chunks will be N-1
    #so if N is odd, then we don't need to adjust the last segment
    
    #define x values to perform the simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #if N is odd or even, we have different
    #numbers of chunks to add
    if((N%2)==0):
        lim = 3
    else:
        lim = 2
        
    #perform the integral using simpsons method
    for i in range(0,len(x)-2,2):
        Fint += simpson_core(f,x[i],h)
        print(i,i+2,x[i],x[i]+h,x[i]+2*h,x[-2],x[-1])
        
    #apply simpsons rule over the last interval
    #if N is even
    if((N%2)==0):
        Fint += simpson_core(f,x[-2],0.5*h)
        
    return Fint


Define the Romberg Integration core

In [ ]:
def romberg_core(f,a,b,I,i):
    
    #we need the difference b-a
    h = b-a
    
    #and the increment between new func evals
    dh = h/2.**(i)
    
    #we ned the cofactor
    K = h/2.**(i+1)
    
    #and the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    #return the answer
    return K*M


Define the wrapper fucntion to perform the Romberg Integration

In [ ]:
def romberg_integration(f,a,b,tol):
    
    #define an iteration variable
    i = 0
    
    #define a maximum number of iterations
    imax = 1000
    
    #define an error estimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        
        #find this romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,I,i)
        
        #compute the new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i] )
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            #iterate
            i += 1
            
            #if we've reached the maximum iterations
            if(i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after ',i)
                
    #return the answer
    return I[i]

In [ ]:
Answer = func_integral(1)-func_integral(0)
print(Answer)
print("Trapezoid")
print(trapezoid_method(func,0,1,10))
print("Simpson's Method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-4
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)